In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [26]:
import pandas as pd
from datetime import datetime
import pickle
import numpy as np
import re
from sklearn.decomposition import PCA
import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.pipeline import Pipeline
# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
# from sklearn.ensemble import
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
# import joblib
import os

# hyperparameter tuning
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV #

from hyperopt import fmin, tpe, hp

import scipy.stats as stats

from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestClassifier # hp tunned
from sklearn.tree import DecisionTreeClassifier # hp tuned
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier #hp tuned
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
# from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

import spacy
import ast
import sklearn
import xgboost

import matplotlib.pyplot as plt

from sklearn.ensemble import VotingClassifier, StackingClassifier

## under sampling
from imblearn.under_sampling import  RandomUnderSampler, EditedNearestNeighbours, CondensedNearestNeighbour, TomekLinks, ClusterCentroids, RepeatedEditedNearestNeighbours, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, AllKNN, InstanceHardnessThreshold

## over sampling
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, KMeansSMOTE, RandomOverSampler, SMOTENC

## over and under sampling
from imblearn.combine import SMOTEENN, SMOTETomek


## ctgan augmentation technique
# from sdv.single_table import CTGANSynthesizer
# from sdv.single_table import GaussianCopulaSynthesizer
# from sdv.single_table import TVAESynthesizer
# from sdv.metadata import SingleTableMetadata
# from sdv.evaluation.single_table import run_diagnostic
# from sdv.evaluation.single_table import evaluate_quality


from sklearn.cluster import MiniBatchKMeans


from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

import spacy
from collections import Counter

# import re



In [3]:
df_train_org =  pd.read_csv('/content/drive/MyDrive/Thesis/data/train/new_11.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/english_test_11_2.csv') # unlabeled data
gold_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/CT24_checkworthy_english_test_gold.tsv',delimiter='\t') # labeled data
dev_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/dev/first_dev_test.csv')
data_to_merge = pd.read_csv('/content/drive/MyDrive/Thesis/data/train/ner_features_with_relevent_noise_prompt.csv')
test_tweet_f = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_falcon_WL.csv')
test_tweet_g = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_gemini_WL.csv')
tweet_test_wo = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/tweet_test_WO_label.csv')

In [8]:
vectorizer_path = '/content/drive/MyDrive/Thesis/models/tfidf_vectorizer_4300.pkl'
tfidf_vect = joblib.load(vectorizer_path)

pca_path = '/content/drive/MyDrive/Thesis/models/pca_356.pkl'
pca = joblib.load(pca_path)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.2 when using version 1.6.1. This might lead to breaking code or invalid 

In [10]:
test_tweet_f.drop('Unnamed: 0', axis = 1, inplace = True)
test_tweet_g.drop('Unnamed: 0', axis = 1, inplace = True)

In [61]:
y_test_f = test_tweet_f['label']
y_test_g = test_tweet_g['label']

In [62]:
print(y_test_f.value_counts())
print(y_test_g.value_counts())

label
0    176
1     59
Name: count, dtype: int64
label
0    128
1    107
Name: count, dtype: int64


In [60]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))


def clean_text(text):
        if not isinstance(text, str):
            text = str(text)
        # remove punctuation
        text = ''.join([char for char in text if char not in string.punctuation]) #
        # tokenize
        tokens = word_tokenize(text)
        # remove stop words
        token = [word for word in tokens if word.lower() not in stop_words]
        return token

test_tweet_f['Tokens'] = test_tweet_f['Text'].apply(clean_text)
test_tweet_g['Tokens'] = test_tweet_g['Text'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
# test_tweet_f.head()

,tweet_id,Text,label,Tokens,cleaned_text
0,1,dr. fauci must be charged for being a pandemic...,0,"[dr, fauci, must, charged, pandemic, planner, ...",dr fauci must charged pandemic planner lied ev...
1,2,you guys remember when the government closed n...,0,"[guys, remember, government, closed, nature, g...",guys remember government closed nature governm...
2,3,the atlantic is trash and nobody believes that...,0,"[atlantic, trash, nobody, believes, pathologic...",atlantic trash nobody believes pathological ly...
3,4,trump chose susan monarez to head the cdc. she...,0,"[trump, chose, susan, monarez, head, cdc, work...",trump chose susan monarez head cdc worked bide...
4,5,the new york times finally admits the conspira...,0,"[new, york, times, finally, admits, conspiracy...",new york times finally admits conspiracy theor...


In [17]:
def cleaned_text(text):
        if not isinstance(text, str):
          text = str(text)
        # lowercase the data
        lower_text = text.lower()
        # removing the punctuations
        remove_punct = ''.join([char for char in lower_text if char not in string.punctuation])
        # remove the stop words
        words = word_tokenize(remove_punct)
        remove_stop_words = [word for word in words if word not in stop_words]
        # removing the numbers
        # removing the extra space
        # replace the repetations of punctuations
        # removing emojis
        # removing emoticons
        # removing contractions

        return ' '.join(remove_stop_words)

test_tweet_g['cleaned_text'] = test_tweet_g['Text'].apply(cleaned_text)
test_tweet_f['cleaned_text'] = test_tweet_f['Text'].apply(cleaned_text)

In [19]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [22]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def get_vader_sentiment_score(text):
    if not isinstance(text, str):
      text = str(text)
    return sia.polarity_scores(text)

test_tweet_f['sentiment_scores'] = test_tweet_f['Text'].apply(get_vader_sentiment_score)
test_tweet_g['sentiment_scores'] = test_tweet_g['Text'].apply(get_vader_sentiment_score)
# extract individual sentiment into seperate columns
test_tweet_f = pd.concat([test_tweet_f.drop(['sentiment_scores'], axis = 1),
                   test_tweet_f['sentiment_scores'].apply(pd.Series)], axis = 1).rename(columns= {'neg' : 'vader_neg', 'neu': 'vader_neu','pos' : 'vader_pos','compound' : 'vader_compound'})

test_tweet_g = pd.concat([test_tweet_g.drop(['sentiment_scores'], axis = 1),
                   test_tweet_g['sentiment_scores'].apply(pd.Series)], axis = 1).rename(columns= {'neg' : 'vader_neg', 'neu': 'vader_neu','pos' : 'vader_pos','compound' : 'vader_compound'})

In [24]:
# test_tweet_f.head()

In [27]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [28]:
def get_sentiment_features(text):
    if not isinstance(text, str):
        text = str(text)
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Perform inference
    outputs = model(**tokens)
    logits = outputs.logits

    # Apply softmax to calculate probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    negative, neutral, positive = probs[0].tolist()

    # Define thresholds for "mixed" classification (adjust as needed)
    if max([positive, negative, neutral]) == positive and positive > 0.5:
        sentiment = "Positive"
    elif max([positive, negative, neutral]) == negative and negative > 0.5:
        sentiment = "Negative"
    else:
        sentiment = "Mixed"

    return {
        "positive_score": positive,
        "negative_score": negative,
        "neutral_score": neutral,
        "sentiment_class": sentiment
    }

In [29]:
test_tweet_f['roberta_sentiment'] = test_tweet_f['Text'].apply(get_sentiment_features)
test_tweet_g['roberta_sentiment'] = test_tweet_g['Text'].apply(get_sentiment_features)

In [30]:
test_tweet_f["positive_score"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["positive_score"])
test_tweet_f["negative_score"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["negative_score"])
test_tweet_f["neutral_score"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["neutral_score"])
test_tweet_f["sentiment_class"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

test_tweet_f["is_positive"] = test_tweet_f["sentiment_class"].apply(lambda x: 1 if x == "Positive" else 0)
test_tweet_f["is_negative"] = test_tweet_f["sentiment_class"].apply(lambda x: 1 if x == "Negative" else 0)
test_tweet_f["is_mixed"] = test_tweet_f["sentiment_class"].apply(lambda x: 1 if x == "Mixed" else 0)


test_tweet_g["positive_score"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["positive_score"])
test_tweet_g["negative_score"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["negative_score"])
test_tweet_g["neutral_score"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["neutral_score"])
test_tweet_g["sentiment_class"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

test_tweet_g["is_positive"] = test_tweet_g["sentiment_class"].apply(lambda x: 1 if x == "Positive" else 0)
test_tweet_g["is_negative"] = test_tweet_g["sentiment_class"].apply(lambda x: 1 if x == "Negative" else 0)
test_tweet_g["is_mixed"] = test_tweet_g["sentiment_class"].apply(lambda x: 1 if x == "Mixed" else 0)

In [31]:
test_tweet_g.head()

,tweet_id,Text,label,Tokens,cleaned_text,vader_neg,vader_neu,vader_pos,vader_compound,roberta_sentiment,positive_score,negative_score,neutral_score,sentiment_class,is_positive,is_negative,is_mixed
0,1,dr. fauci must be charged for being a pandemic...,1,"[dr, fauci, must, charged, pandemic, planner, ...",dr fauci must charged pandemic planner lied ev...,0.173,0.827,0.000,-0.5267,"{'positive_score': 0.0036742433439940214, 'neg...",0.003674,0.943537,0.052788,Negative,0,1,0
1,2,you guys remember when the government closed n...,1,"[guys, remember, government, closed, nature, g...",guys remember government closed nature governm...,0.000,0.962,0.038,0.1695,"{'positive_score': 0.043167684227228165, 'nega...",0.043168,0.649833,0.306999,Negative,0,1,0
2,3,the atlantic is trash and nobody believes that...,0,"[atlantic, trash, nobody, believes, pathologic...",atlantic trash nobody believes pathological ly...,0.290,0.634,0.076,-0.9022,"{'positive_score': 0.002745778299868107, 'nega...",0.002746,0.963693,0.033561,Negative,0,1,0
3,4,trump chose susan monarez to head the cdc. she...,1,"[trump, chose, susan, monarez, head, cdc, work...",trump chose susan monarez head cdc worked bide...,0.058,0.709,0.233,0.8553,"{'positive_score': 0.05480159819126129, 'negat...",0.054802,0.706653,0.238545,Negative,0,1,0
4,5,the new york times finally admits the conspira...,1,"[new, york, times, finally, admits, conspiracy...",new york times finally admits conspiracy theor...,0.076,0.874,0.049,-0.2960,"{'positive_score': 0.03213520720601082, 'negat...",0.032135,0.700300,0.267565,Negative,0,1,0


In [32]:
def count_tokens(tokens):
    return len(tokens)
test_tweet_f['token_length'] = test_tweet_f['Tokens'].apply(count_tokens)
test_tweet_g['token_length'] = test_tweet_g['Tokens'].apply(count_tokens)

In [33]:
nlp = spacy.load("en_core_web_sm")

In [34]:
def count_entities_to_columns(text):


    # # Process each row in the DataFrame
    # for text in df[text_column]:
    #     # Default counts for each row
    names = 0
    orgs = 0
    dates = 0

    if not isinstance(text, str):
        text = str(text)

    # Process text using spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            names += 1
        elif ent.label_ == "ORG":
            orgs += 1
        elif ent.label_ == "DATE":
            dates += 1

    return {
        "names": names,
        "organizations": orgs,
        "dates": dates
    }

In [35]:
test_tweet_g['counts'] = test_tweet_g['Text'].apply(count_entities_to_columns)
test_tweet_f['counts'] = test_tweet_f['Text'].apply(count_entities_to_columns)

In [36]:
test_tweet_g["names"] = test_tweet_g["counts"].apply(lambda x: x["names"])
test_tweet_g["organizations"] = test_tweet_g["counts"].apply(lambda x: x["organizations"])
test_tweet_g["dates"] = test_tweet_g["counts"].apply(lambda x: x["dates"])

test_tweet_f["names"] = test_tweet_f["counts"].apply(lambda x: x["names"])
test_tweet_f["organizations"] = test_tweet_f["counts"].apply(lambda x: x["organizations"])
test_tweet_f["dates"] = test_tweet_f["counts"].apply(lambda x: x["dates"])

In [38]:
def count_punctuation(text):
    if not isinstance(text, str):
        text = str(text)
    # Define a regular expression pattern to match punctuation characters
    punctuation_pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
    # Find all punctuation marks in the text
    punctuation_count = len(re.findall(punctuation_pattern, text))
    return punctuation_count

In [39]:
test_tweet_f['punctuation_count'] = test_tweet_f['Text'].apply(count_punctuation)
test_tweet_g['punctuation_count'] = test_tweet_g['Text'].apply(count_punctuation)

In [40]:
test_tweet_f.columns

Index(['tweet_id', 'Text', 'label', 'Tokens', 'cleaned_text', 'vader_neg',
       'vader_neu', 'vader_pos', 'vader_compound', 'roberta_sentiment',
       'positive_score', 'negative_score', 'neutral_score', 'sentiment_class',
       'is_positive', 'is_negative', 'is_mixed', 'token_length', 'counts',
       'names', 'organizations', 'dates', 'punctuation_count'],
      dtype='object')

In [41]:
def text_length(text):
  if not isinstance(text, str):
    text = str(text)
  return len(text)

In [44]:
test_tweet_f['text_length'] = test_tweet_f['Text'].apply(text_length)
test_tweet_g['text_length'] = test_tweet_g['Text'].apply(text_length)

test_tweet_f['cleaned_text_length'] = test_tweet_f['cleaned_text'].apply(text_length)
test_tweet_g['cleaned_text_length'] = test_tweet_g['cleaned_text'].apply(text_length)

In [48]:
test_tweet_f.columns

Index(['tweet_id', 'Text', 'label', 'Tokens', 'cleaned_text', 'vader_neg',
       'vader_neu', 'vader_pos', 'vader_compound', 'roberta_sentiment',
       'roberta_sent_pos', 'roberta_sent_neg', 'roberta_sent_mixed',
       'sentiment_class', 'is_positive', 'is_negative', 'is_mixed',
       'count_tokens', 'counts', 'names', 'organizations', 'dates',
       'punctuation_count', 'text_length', 'cleaned_text_length'],
      dtype='object')

In [47]:
test_tweet_f.rename(columns = {'raw_text_length': 'text_length', 'negative_score': 'roberta_sent_neg', 'neutral_score': 'roberta_sent_mixed', 'positive_score': 'roberta_sent_pos', 'token_length': 'count_tokens'}, inplace = True)

test_tweet_g.rename(columns = {'raw_text_length': 'text_length', 'negative_score': 'roberta_sent_neg', 'neutral_score': 'roberta_sent_mixed', 'positive_score': 'roberta_sent_pos', 'token_length': 'count_tokens'}, inplace = True)

In [ ]:
test_tweet_g.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_g.csv', index = False)
test_tweet_f.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_f.csv', index = False)